# Projekt MAGN

### Autorzy:

- Szymon Jurecki
- Dominik Breksa

In [18]:
%pip install .
%pip install pandas
%pip install numpy

Processing c:\users\domin\pycharmprojects\ggsn---magn
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for magn: filename=magn-1.0.0.0-py3-none-any.whl size=9937 sha256=2b80fc024412c002b040cf57159ba06cc6a43434a6c6ff47dfeada08ee0fe59a
  Stored in directory: c:\users\domin\appdata\local\pip\cache\wheels\02\69\7a\7a321fc99cdf850f17c5dbe96e53ef1e4b07adda9d5bd3d1f7
Successfully built magn
  Attempting uninstall: magn
    Found existing installation: magn 1.0.0.0
    Not uninstalling magn at c:\users\domin\pycharmprojects\ggsn---magn\src, outside environment C:\Users\domin\PycharmProjects\GGSN---

DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Pobranie oraz Przygotowanie Danych

In [19]:
from pathlib import Path
from typing import Final

resources_dir: Final[Path] = Path('resources/data/')
zip_data_file: Final[Path] = resources_dir.joinpath('pitchfork-data.zip')
database_path: Final[Path]  = resources_dir.joinpath('database.sqlite')

Pobranie danych z platformy Kaggle

[Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/nolanbconaway/pitchfork-data)

In [20]:
from os.path import exists

if not exists(zip_data_file):
    !kaggle datasets download -d nolanbconaway/pitchfork-data -p resources/data

Rozpakowanie danych do odpowiedniego katalogu

In [21]:
from zipfile import ZipFile

if not exists(database_path):
    with ZipFile(zip_data_file, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

Dodanie kluczy głównych i obcych do bazy danych, ponieważ nie zostały one dodane podczas tworzenia na platformie Kaggle, a jest to ważne podczas tworzenia MAGN.

In [22]:
from typing import List, Tuple
import sqlite3

def get_table_columns(db_cursor: sqlite3.Cursor, table_name: str) -> List[Tuple[str, str]]:
    """Returns a list of tuples containing the column name and data type of given table in a database."""
    
    return list(map(
        lambda x: (x[1], x[2]),
        db_cursor.execute(f"""
            PRAGMA
                table_info({table_name});
        """).fetchall(),
    ))

def delete_duplicates(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Deletes duplicates from a given table column in a database."""
    
    db_cursor.execute(f"""
        DELETE FROM
            {table_name}
        WHERE
            {column_name} IN (
                SELECT
                    {column_name}
                FROM
                    {table_name}
                GROUP BY
                    {column_name}
                HAVING
                    COUNT(*) > 1
            );
    """)

def create_table_with_primary_key(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str, column_name: str) -> None:
    """Creates a table with a primary key in a database."""
    
    table_columns = get_table_columns(db_cursor, table_name)
    
    db_cursor.execute(f"""
        CREATE TABLE {temporary_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    if column != column_name
                    else f'{column} {data_type} PRIMARY KEY'
                    for column, data_type in table_columns
                )
            }
        );
    """)
    
def move_data_and_switch_tables(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str) -> None:
    """Moves data from a temporary table to the original table and switches the tables in a database."""
    
    db_cursor.executescript(f"""
        -- Move the data
        INSERT INTO
            {temporary_table_name}
        SELECT
            *
        FROM
            {table_name};
        
        -- Drop the old table
        DROP TABLE
            {table_name};
        
        -- Rename the new table
        ALTER TABLE
            {temporary_table_name}
        RENAME TO
            {table_name};
    """)

def add_primary_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Adds a primary key to a given table column in a database."""
    temporary_table_name = f'{table_name}_'
    
    delete_duplicates(db_cursor, table_name, column_name)
    create_table_with_primary_key(db_cursor, table_name, temporary_table_name, column_name)
    move_data_and_switch_tables(db_cursor, table_name, temporary_table_name)

def create_table_with_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str, temporary_foreign_table_name: str) -> None:
    """Creates a table with a foreign key in a database."""

    foreign_table_columns = get_table_columns(db_cursor, foreign_table_name)
    
    db_cursor.execute(f"""
        CREATE TABLE {temporary_foreign_table_name} (
            {
            ', '.join(
                f'{column} {data_type}'
                for column, data_type in foreign_table_columns
            )
            },
            CONSTRAINT fk_{foreign_table_name}_{foreign_column_name}
                FOREIGN KEY
                    ({foreign_column_name})
                REFERENCES
                    {table_name} ({column_name})
        );
    """)

def add_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str) -> None:
    """Adds a foreign key to a given table column in a database."""

    temporary_foreign_table_name = f'{foreign_table_name}_'
        
    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = OFF;
    """)
    
    create_table_with_foreign_key(db_cursor, table_name, column_name, foreign_table_name, foreign_column_name, temporary_foreign_table_name)
    move_data_and_switch_tables(db_cursor, foreign_table_name, temporary_foreign_table_name)

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = ON;
    """)

In [23]:
with sqlite3.connect(database_path) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'reviews', 'reviewid')

        add_foreign_key(cursor, 'reviews', 'reviewid', 'artists', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'content', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'genres', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'labels', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'years', 'reviewid')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

![ERD Diagram for the database schema](docs/images/database_erd.png)

In [24]:
from magn.database.sqlite3 import get_table_names

all_tables = get_table_names(database_path)

all_tables

['reviews', 'artists', 'content', 'genres', 'labels', 'years']

In [25]:
from magn.database.sqlite3 import SQLite3KeysReader

keys_reader = SQLite3KeysReader(database_path, all_tables)
keys_reader.read()

{'reviews': Keys(primary_keys=['reviewid'], foreign_keys={}),
 'artists': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'content': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'genres': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'labels': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'years': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')})}

In [26]:
from magn.database.sqlite3 import SQLite3DataReader

data_reader = SQLite3DataReader(database_path, all_tables, keys_reader.read())
data_reader.read()

{'reviews':                                                       title           artist  \
 reviewid                                                                       
 1                                             young forever        aberfeldy   
 6                                      pure tone audiometry         aarktica   
 7         or you could just go through your whole life a...         aarktica   
 8                                            bleeding light         aarktica   
 10                                                     abcs             abcs   
 ...                                                     ...              ...   
 22721                                          prelapsarian         krallice   
 22722         insecure (music from the hbo original series)  various artists   
 22724                          filthy america its beautiful          the lox   
 22725                                             new start             taso   
 22745           

In [27]:
dfs = data_reader.read()

dfs[0]

KeyError: 0

In [ ]:
dfs[1]

In [ ]:
from magn.database.database import Database

dataframe_database = Database.from_sqlite3(database_path)

In [ ]:
from magn.database.topological_sort import TopologicalSorter

sort_data = {
    '0': ['1'],
    '1': ['2'],
    '2': [],
    '3': ['1', '2']
}

topological_sort = TopologicalSorter()
for x in topological_sort.sort(sort_data):
    print(x)